# Module 7: The "Zero to Hero" Deployment Pipeline

This is the final integration. We combine Physics (ZNE) with AI to create a hybrid solver that beats the baseline.

## 7.1 The Hybrid Architecture

1.  **ZNE Layer:** Run Exponential ZNE to get a physics-informed baseline.
2.  **AI Correction:** Feed the circuit sequence into our pre-trained LSTM (from Module 6).
3.  **Final Result:** `Prediction = ZNE_Estimate + AI_Residual`

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from qiskit_aer import AerSimulator
from qiskit import transpile
import utils

# --- 1. Define Model Architecture (Must match Module 6) ---
class QEM_LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=16, hidden_size=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        last_out = lstm_out[:, -1, :] 
        return self.fc(last_out)

# --- 2. PIPELINE DEFINITION ---
class HackathonPipeline:
    def __init__(self, model_path="qem_lstm.pth"):
        self.tokenizer = utils.CircuitTokenizer(max_length=60)
        
        # Load AI
        try:
            vocab_size = len(self.tokenizer.vocab) + 1
            self.model = QEM_LSTM(vocab_size)
            self.model.load_state_dict(torch.load(model_path))
            self.model.eval()
            print("✅ AI Model Loaded Successfully.")
        except FileNotFoundError:
            print("⚠️ Model file not found. Please run Module 6 first!")
            self.model = None

    def run_zne(self, qc, simulator=None):
        """
        Runs Exponential ZNE using shared logic.
        """
        scales = [1.0, 2.0, 3.0]
        results = []
        
        for s in scales:
            # Build noisy simulator for this scale
            nm = utils.build_noise_model(scale=s)
            sim = AerSimulator(noise_model=nm)
            
            qc_t = transpile(qc, sim)
            counts = sim.run(qc_t, shots=1000).result().get_counts()
            shots = sum(counts.values())
            val = (counts.get('00', 0)+counts.get('11', 0) - counts.get('01', 0)-counts.get('10', 0))/shots
            results.append(val)
            
        # Extrapolate
        def exp_decay(x, a, b, c): return a * np.exp(-b * x) + c
        try:
            popt, _ = curve_fit(exp_decay, scales, results, p0=[1, 0.1, 0], maxfev=2000)
            return exp_decay(0, *popt)
        except:
            lr = LinearRegression()
            lr.fit(np.array(scales).reshape(-1, 1), results)
            return lr.predict([[0.0]])[0]

    def predict(self, target_circuit, instructions):
        # 1. ZNE Baseline
        base_estimate = self.run_zne(target_circuit)
        
        if self.model is None:
            return base_estimate, 0.0
            
        # 2. AI Correction
        # Tokenize instructions
        seq = self.tokenizer.tokenize(instructions)
        seq_t = torch.tensor([seq], dtype=torch.long)
        
        with torch.no_grad():
            predicted_residual = self.model(seq_t).item()
            
        return base_estimate + predicted_residual, predicted_residual, base_estimate

# --- 3. BENCHMARK SUITE ---
print("Initializing Hybrid Pipeline...")
pipeline = HackathonPipeline()

def evaluate_circuit(name, qc, instructions, verbose=True):
    qc.measure_all()
    
    # Get Truth (Simulate with NO Noise)
    sim_ideal = AerSimulator(method='stabilizer') # Works for Clifford, approx for others
    # Note: For non-Clifford (Variational/QAOA), stabilizer method fails. 
    # We automatically switch to statevector if needed, but for speed we keep circuits small.
    try:
        res = sim_ideal.run(transpile(qc, sim_ideal), shots=1000).result().get_counts()
    except:
        # Fallback for non-Clifford
        sim_ideal = AerSimulator(method='statevector')
        res = sim_ideal.run(transpile(qc, sim_ideal), shots=1000).result().get_counts()
        
    shots = sum(res.values())
    true_val = (res.get('00',0)+res.get('11',0) - res.get('01',0)-res.get('10',0))/shots

    final_pred, ai_res, zne_base = pipeline.predict(qc, instructions)
    
    err_base = abs(true_val - zne_base)
    err_model = abs(true_val - final_pred)
    
    # Improvement Ratio R = E_base / E_model
    # R > 1 means Model is better.
    # Handle division by zero
    if err_model < 1e-5: ratio = 100.0 # Cap at 100x
    else: ratio = err_base / err_model

    if verbose:
        print(f"\n--- Benchmarking: {name} ---")
        print(f"True Value:     {true_val:.4f}")
        print(f"ZNE Baseline:   {zne_base:.4f} (Err: {err_base:.4f})")
        print(f"Hybrid Model:   {final_pred:.4f} (Err: {err_model:.4f})")
        print(f"Improvement Ratio (R): {ratio:.2f}x")
        
    return err_base, err_model, ratio

# A. Random Clifford (Scaling Test)
print("\n=== SCALING TEST (Random Circuits) ===")
depths = [5, 15, 25, 35]
for d in depths:
    qc, instr = utils.create_random_clifford_circuit(2, d)
    evaluate_circuit(f"Random Clifford (Depth {d})", qc, instr)

# B. Variational Ansatz
print("\n=== VARIATIONAL TEST ===")
qc_var, instr_var = utils.create_variational_circuit(2, 5)
evaluate_circuit("Hardware Efficient Ansatz", qc_var, instr_var)

# C. QAOA Layer
print("\n=== QAOA TEST ===")
qc_qaoa, instr_qaoa = utils.create_qaoa_circuit(2, p=2)
evaluate_circuit("QAOA (p=2)", qc_qaoa, instr_qaoa)